In [3]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:3


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [4]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
n_value = np.array([1.0,3.0,5.0,8.0,10.0]).reshape(-1,1)
r_value = np.array([2,6,8]).reshape(-1,1)

LR_tune,N_value,R_value = np.meshgrid(lr_tune,n_value,r_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
N_value = N_value.flatten('F').reshape(-1,1)
R_value = R_value.flatten('F').reshape(-1,1)


lrnr_tune = np.hstack((LR_tune,N_value,R_value))

In [6]:
def true_1D_2(x): #True function for 1D_1 dy/dx = cos(0.01*x) BC1: y(0)=0; x \in [-100,100]
    y = 100*np.sin(0.01*x)
    return y
    

In [7]:
loss_thresh = 0.005

x = np.linspace(-600,600,5000).reshape(-1,1)
ysol = true_1D_2(x)

bc1_x = np.array(0).reshape(-1,1) 
bc1_y = np.array(0).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)

 
x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)

y_true = true_1D_2(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

#torch.autograd.set_detect_anomaly(True)

In [8]:
def colloc_pts(N_f,seed):

  #Collocation Points
  # Latin Hypercube sampling for collocation points 
  # N_f sets of tuples(x,y)
  x01 = np.array([[0.0, 1.0]])
  sampling = LHS(xlimits=x01,random_state =seed)

  x_coll_train = lb + (ub-lb)*sampling(N_f)
  x_coll_train = np.vstack((x_coll_train, bc1_x)) # append training points to collocation points 

  return x_coll_train

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        self.omega1.requiresGrad = True
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.alpha.requiresGrad = True
        
        self.omega = Parameter((1/n_val)*torch.ones(rowdy_terms,len(layers)-2))
        self.omega.requiresGrad = True
        
        self.n = torch.tensor(n_val)

                      
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        f = dy_dx - torch.cos(0.01*g)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + 100*loss_f
        
        return loss_val
     
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred

    def test_loss(self):
        y_pred = self.test()
        
       
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
      
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(seed):
    x_coll_np_array = colloc_pts(N_f,seed*123)
    x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_coll_train,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0

    x_coll = torch.from_numpy(colloc_pts(N_f,123)).float().to(device)
    f_hat = torch.zeros(x_coll.shape[0],1).to(device)

    nan_flag = 0      
    for i in range(max_iter):
        train_step(i)
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break

    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))
    
    return nan_flag

In [13]:
nan_tune = []
for tune_reps in range(75):
    label = "1D_FODE_rowdy_tune"+str(tune_reps) 
    max_reps = 10
    max_iter = 100

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    alpha_full = []
    omega_full = []
    elapsed_time= np.zeros((max_reps,1))

    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))
  
    n_val = lrnr_tune[tune_reps,1]
    rowdy_terms = int(lrnr_tune[tune_reps,2])

    for reps in range(max_reps):
        train_loss = []
        test_mse_loss = []
        test_re_loss = []  
        alpha_val = []
        omega_val = []
      
        torch.manual_seed(reps*36)
        N_f = 10000 #Total number of collocation points
    
        layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
        PINN = Sequentialmodel(layers,n_val,rowdy_terms)
      
    
        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())
      
        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrnr_tune[tune_reps,0], 
                                max_iter = 10, 
                                max_eval = 15, 
                                tolerance_grad = 1e-6, 
                                tolerance_change = 1e-6, 
                                history_size = 100, 
                                line_search_fn = 'strong_wolfe')
    

      
        nan_flag = train_model(max_iter,reps)

      
        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        alpha_full.append(alpha_val)
        omega_full.append(omega_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break
      
      
        #print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full,"omega": omega_full, "label": label}
    savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 47.76848 Test MSE 5220.120268452789 Test RE 0.9997546187042216
1 Train Loss 47.76816 Test MSE 5219.909834805189 Test RE 0.9997344674317818
2 Train Loss 47.76797 Test MSE 5219.475488846356 Test RE 0.9996928728762264
3 Train Loss 47.751198 Test MSE 5185.030696847399 Test RE 0.996388784980

95 Train Loss 0.07076411 Test MSE 3.2856236371641687 Test RE 0.02508199470276381
96 Train Loss 0.07076411 Test MSE 3.2856236371641687 Test RE 0.02508199470276381
97 Train Loss 0.07076411 Test MSE 3.2856236371641687 Test RE 0.02508199470276381
98 Train Loss 0.07076411 Test MSE 3.2856236371641687 Test RE 0.02508199470276381
99 Train Loss 0.07076411 Test MSE 3.2856236371641687 Test RE 0.02508199470276381
Training time: 36.56
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
 

86 Train Loss 0.009701246 Test MSE 0.05375906589061922 Test RE 0.003208329921481089
87 Train Loss 0.009649175 Test MSE 0.06841166823741708 Test RE 0.003619247267225289
88 Train Loss 0.009559386 Test MSE 0.08001835267929103 Test RE 0.00391424466108391
89 Train Loss 0.009487702 Test MSE 0.08908549808022048 Test RE 0.0041300629475971755
90 Train Loss 0.009403443 Test MSE 0.10099337226674733 Test RE 0.0043974367510620236
91 Train Loss 0.009313456 Test MSE 0.0819979192164757 Test RE 0.003962365928605443
92 Train Loss 0.009233563 Test MSE 0.055121973426593446 Test RE 0.003248744396090803
93 Train Loss 0.009149269 Test MSE 0.03589875258622675 Test RE 0.0026217594594851045
94 Train Loss 0.00909898 Test MSE 0.028583755107978937 Test RE 0.002339444819108841
95 Train Loss 0.009065926 Test MSE 0.028842799033798314 Test RE 0.002350021668152326
96 Train Loss 0.009010803 Test MSE 0.029126048843411207 Test RE 0.0023615326337282413
97 Train Loss 0.008910426 Test MSE 0.02477147394927494 Test RE 0.002177

79 Train Loss 0.032749068 Test MSE 4.886570035025041 Test RE 0.030588292075279792
80 Train Loss 0.032232672 Test MSE 4.4152179981087025 Test RE 0.029075637146771662
81 Train Loss 0.030151693 Test MSE 3.2562683846579703 Test RE 0.024969696347751404
82 Train Loss 0.023964971 Test MSE 0.7874668843167824 Test RE 0.012279178546686045
83 Train Loss 0.022444012 Test MSE 0.25056749809975193 Test RE 0.006926527024184756
84 Train Loss 0.02134325 Test MSE 0.2666603320568914 Test RE 0.007145495868743123
85 Train Loss 0.020123446 Test MSE 0.32819686413302246 Test RE 0.007927210273520092
86 Train Loss 0.018551813 Test MSE 0.48620294632691113 Test RE 0.009648547913056941
87 Train Loss 0.018208506 Test MSE 0.40345261368960217 Test RE 0.008789201705824923
88 Train Loss 0.017737757 Test MSE 0.28944673892121475 Test RE 0.007444533612707712
89 Train Loss 0.016259894 Test MSE 0.22097720397286894 Test RE 0.0065046944622795
90 Train Loss 0.014330288 Test MSE 0.7110397343500019 Test RE 0.011668098857645129
91

73 Train Loss 0.10558736 Test MSE 96.16640413444125 Test RE 0.13569531491826078
74 Train Loss 0.10311912 Test MSE 92.92478408926998 Test RE 0.1333886708410537
75 Train Loss 0.10200098 Test MSE 92.98993889048062 Test RE 0.13343542580276463
76 Train Loss 0.09987345 Test MSE 93.922795905341 Test RE 0.13410305474406717
77 Train Loss 0.09795497 Test MSE 91.1916613315708 Test RE 0.13213891259368135
78 Train Loss 0.09625001 Test MSE 90.12122436182945 Test RE 0.1313610787705443
79 Train Loss 0.093938835 Test MSE 92.67657503375295 Test RE 0.13321040615861096
80 Train Loss 0.09283349 Test MSE 94.49921223739715 Test RE 0.13451392919657365
81 Train Loss 0.092231646 Test MSE 94.9771505158658 Test RE 0.13485365834266658
82 Train Loss 0.09155852 Test MSE 94.63103866848425 Test RE 0.13460771998540566
83 Train Loss 0.090912186 Test MSE 94.05641083490835 Test RE 0.13419840860335208
84 Train Loss 0.09020636 Test MSE 94.12829086568625 Test RE 0.13424967753547937
85 Train Loss 0.08859479 Test MSE 91.324884

...

0 Train Loss 47.699768 Test MSE 5201.0388219050665 Test RE 0.9979257116669747
1 Train Loss 47.575928 Test MSE 5192.836743515333 Test RE 0.9971385329206462
2 Train Loss 47.168056 Test MSE 5174.156675632396 Test RE 0.995343425749761
3 Train Loss 46.731117 Test MSE 5147.822107382966 Test RE 0.9928072272473767
4 Train Loss 46.228672 Test MSE 5112.106850620745 Test RE 0.9893572166861888
5 Train Loss 45.80153 Test MSE 5076.51393529679 Test RE 0.985907013343687
6 Train Loss 45.147255 Test MSE 5010.13130785386 Test RE 0.9794397345535466
7 Train Loss 44.603058 Test MSE 4970.86214092027 Test RE 0.975593782992863
8 Train Loss 43.698044 Test MSE 4879.198355578068 Test RE 0.9665568469803432
9 Train Loss 43.077175 Test MSE 4851.4456315232765 Test RE 0.9638040547473986
10 Train Loss 42.22734 Test MSE 4790.130993554634 Test RE 0.9576942053959469
11 Train Loss 41.418716 Test MSE 4684.687099382168 Test RE 0.9470948150357823
12 Train Loss 40.497635 Test MSE 4609.8027891553465 Test RE 0.9394947073649826
1

0 Train Loss nan Test MSE nan Test RE nan
NAN BREAK!
Training time: 2.42
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 47.761093 Test MSE 5222.048794609697 Test RE 0.9999392767955084
1 Train Loss 47.612225 Test MSE 5207.9922506727235 Test RE 0.9985925676447551
2 Train Loss 47.612225 Test MSE 5207.9922506727235 Test RE 0.99859256764

91 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
92 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
93 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
94 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
95 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
96 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
97 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
98 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
99 Train Loss 0.0033146318 Test MSE 0.041898090870957846 Test RE 0.0028323719310325663
Training time: 99.63
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_feature

86 Train Loss 0.018189289 Test MSE 3.334512895844182 Test RE 0.025267912560434803
87 Train Loss 0.018145025 Test MSE 3.3035870826450306 Test RE 0.02515046646611621
88 Train Loss 0.017888006 Test MSE 3.0697074737702454 Test RE 0.024243854028168324
89 Train Loss 0.016707474 Test MSE 3.146253357495991 Test RE 0.0245442638352489
90 Train Loss 0.013519505 Test MSE 3.164009778714701 Test RE 0.02461342627433642
91 Train Loss 0.011791379 Test MSE 2.8790017105135077 Test RE 0.023478704297244257
92 Train Loss 0.01065338 Test MSE 1.4519858856872145 Test RE 0.016673791329393436
93 Train Loss 0.010015438 Test MSE 0.9880736323009996 Test RE 0.013754595392050152
94 Train Loss 0.009930821 Test MSE 1.1446182285856543 Test RE 0.014804150573360635
95 Train Loss 0.009758596 Test MSE 1.0638305984430576 Test RE 0.014272150065053772
96 Train Loss 0.009665883 Test MSE 1.0744304878586868 Test RE 0.014343076881864189
97 Train Loss 0.009043047 Test MSE 0.32643198049804867 Test RE 0.007905867194674107
98 Train Lo

79 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
80 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
81 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
82 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
83 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
84 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
85 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
86 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
87 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
88 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
89 Train Loss 0.0013016311 Test MSE 0.020695100075768334 Test RE 0.0019906134315188667
90 Train Loss 0.0013016311 Test MSE 0.02069

In [14]:
label

'1D_FODE_rowdy_tune74'

In [15]:
#3,4,8,9,13,14,18,19,23,24

In [16]:
import scipy.io as sio

In [19]:
for tune_reps in range(75):
#  if tune_reps not in s:
    label = "1D_FODE_rowdy_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   0.06168178036858476
1   [[1.00040899 1.00037159 1.00004399 1.00004294 1.00000428 0.99999969
  0.99998701 0.99995316 0.99987398 0.99232713        nan]]
2   0.07911200988304705
3   0.0524902091360661
4   0.06291599062512308
5   0.056747154816179105
6   0.0035422761936902887
7   [[1.00010564 0.99869085 0.99154167        nan]]
8   [[1.00008686 0.99999219 0.9996634  0.99414351 0.97722639 0.93397531
  0.91532785        nan]]
9   [[0.99625671 0.99361166 0.98748302        nan]]
10   [[0.99957558 0.99923617 0.99516374 0.99085265 0.96115865        nan]]
11   [[1.00040519 1.00020124 0.99998713 0.99881668        nan]]
12   0.10766062129795857
13   [[1.0000447       nan]]
14   [[0.98785103        nan]]
15   [[0.99979404 0.99538945 0.98275113 0.97586583        nan]]
16   [[0.99931011 0.99331053 0.97350887        nan]]
17   [[0.99358986        nan]]
18   [[0.99157495 0.99157495 0.99157495 0.99157495        nan]]
19   [[0.99164121        nan]]
20   nan
21   [[0.99343178 0.98132754        nan]]
22 

In [7]:
lrnr_tune[3]

array([0.05, 8.  , 2.  ])